In [42]:
from importlib.metadata import version
import re
import os
import urllib.request
from pathlib import Path
from urllib.parse import urlparse

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.8.0
tiktoken version: 0.11.0


In [44]:
urls = [
    "https://www.gutenberg.org/cache/epub/32854/pg32854.txt",
    "https://www.gutenberg.org/cache/epub/19657/pg19657.txt",
    "https://www.gutenberg.org/cache/epub/17489/pg17489.txt"
]

data_folder = Path("./data")
data_folder.mkdir(exist_ok=True)  # crée le dossier si nécessaire

for url in urls:
    filename = Path(urlparse(url).path).name   # récupère le nom du fichier depuis l'URL
    file_path = data_folder / filename        # chemin complet vers le fichier

    if not file_path.exists():
        print(f"Téléchargement de {filename}...")
        urllib.request.urlretrieve(url, file_path)
    else:
        print(f"{filename} existe déjà, téléchargement ignoré.")

pg32854.txt existe déjà, téléchargement ignoré.
pg19657.txt existe déjà, téléchargement ignoré.
Téléchargement de pg17489.txt...


In [45]:
raw_text = ""

# Liste des fichiers dans ./data correspondant aux URLs
file_paths = [data_folder / Path(urlparse(url).path).name for url in urls]

# Lire tous les fichiers et les concaténer avec le marqueur
texts = []
for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        texts.append(f.read())

# Concatène avec le marqueur, sans le mettre au début
raw_text = " <|endoftext|> ".join(texts)

print("Total number of characters:", len(raw_text))
#print(raw_text[:99])

Total number of characters: 2098400


In [46]:
regex = r'([,.:;?_!"()\']|--|\s)'
result = re.split(regex, raw_text)
result = [item for item in result if item.strip()]

In [47]:
all_words = sorted(set(result))
if "<|endoftext|>" not in all_words:
    all_words.extend(["<|endoftext|>"])
if "<|unk|>" not in all_words:
    all_words.extend(["<|unk|>"])
vocab_size = len(all_words)

print(f"Mon vocabulaire contient {vocab_size} mots.")

Mon vocabulaire contient 28699 mots.


In [48]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [50]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [51]:
tokenizer = SimpleTokenizerV2(vocab)


ids = tokenizer.encode(raw_text)
print(ids)

[28697, 3038, 1794, 11774, 18787, 388, 2257, 13, 14267, 3764, 11785, 16222, 13645, 25478, 26338, 18787, 5415, 5417, 15505, 25478, 3900, 3643, 5324, 18010, 19071, 19441, 18787, 25478, 27135, 5992, 18449, 9430, 5324, 27129, 5144, 18449, 22660, 27118, 17, 4091, 17506, 9348, 16231, 13, 14309, 16231, 6377, 18939, 21847, 16231, 26297, 25478, 25432, 18787, 25478, 3038, 1794, 2196, 15561, 27129, 25486, 11785, 18939, 18888, 5992, 27138, 17, 14686, 17, 18980, 17, 1956, 27143, 5707, 18524, 16856, 15505, 25478, 3900, 3643, 13, 27143, 27127, 14917, 25578, 8113, 25478, 16645, 18787, 25478, 9494, 27121, 27143, 5707, 16856, 6661, 26345, 25486, 11774, 17, 3791, 182, 388, 2257, 13, 14267, 445, 182, 2414, 2166, 1000, 182, 2082, 961, 3209, 10049, 182, 2086, 63, 13, 111, 4340, 4, 2544, 21883, 26326, 182, 431, 125, 13, 111, 2157, 182, 1609, 1096, 182, 3036, 7300, 278, 2107, 17, 1818, 26595, 20766, 7300, 933, 1758, 17, 12, 3348, 2688, 3695, 2793, 1644, 1338, 205, 2124, 13, 1633, 12, 278, 2107, 3839, 8, 23570

In [53]:
tokenizer = SimpleTokenizerV2(vocab)

text = "Bonjour, Comment t'appelles-tu ?"

tokenizer.decode(tokenizer.encode(text))


"Bonjour, Comment t' appelles-tu?"

In [55]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))
enc_sample = enc_text[100:150]

460981


In [56]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [3791, 182, 388, 2257]
y:      [182, 388, 2257, 13]


In [57]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[3791] ----> 182
[3791, 182] ----> 388
[3791, 182, 388] ----> 2257
[3791, 182, 388, 2257] ----> 13


In [58]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

Title ----> :
Title: ----> Arsène
Title: Arsène ----> Lupin
Title: Arsène Lupin ----> ,


In [59]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.8.0


/Users/simon/Documents/my_1st_llm/venv/lib/python3.13/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [64]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt)
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [65]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = SimpleTokenizerV2(vocab)

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [66]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[28697,  3038,  1794, 11774]]), tensor([[ 3038,  1794, 11774, 18787]])]


In [67]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[28697,  3038,  1794, 11774],
        [18787,   388,  2257,    13],
        [14267,  3764, 11785, 16222],
        [13645, 25478, 26338, 18787],
        [ 5415,  5417, 15505, 25478],
        [ 3900,  3643,  5324, 18010],
        [19071, 19441, 18787, 25478],
        [27135,  5992, 18449,  9430]])

Targets:
 tensor([[ 3038,  1794, 11774, 18787],
        [  388,  2257,    13, 14267],
        [ 3764, 11785, 16222, 13645],
        [25478, 26338, 18787,  5415],
        [ 5417, 15505, 25478,  3900],
        [ 3643,  5324, 18010, 19071],
        [19441, 18787, 25478, 27135],
        [ 5992, 18449,  9430,  5324]])


In [68]:
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
max_length = 4 # taille de contexte -> faible ici - pour un vrai modèle, on prendra une taille plus grande
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [70]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[28697,  3038,  1794, 11774],
        [18787,   388,  2257,    13],
        [14267,  3764, 11785, 16222],
        [13645, 25478, 26338, 18787],
        [ 5415,  5417, 15505, 25478],
        [ 3900,  3643,  5324, 18010],
        [19071, 19441, 18787, 25478],
        [27135,  5992, 18449,  9430]])

Inputs shape:
 torch.Size([8, 4])


In [71]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)


torch.Size([8, 4, 256])


In [72]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [73]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)